In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
import tensorflow as tf

StyleTransferLayer.ipynb load_data.ipynb          test.jpg
Untitled.ipynb           main_tf.py               train
__pycache__              read_data.py
basic_block.py           test


In [2]:
vgg = tf.keras.applications.vgg19.VGG19(include_top=False, weights='imagenet')

80142336/80134624 [==============================] - 10s 0us/step
